In [ ]:
import numpy as np
import simulation_manager, results_annalysis
import matplotlib.pyplot as plt
import time
import pandas as pd

In [ ]:
def P_aug(N,bit): 
    P = np.zeros((N,N*bit))
    b = np.array([2**i for i in range(bit)])
    for i in range(N):
        debut = i*bit
        fin = debut + bit 
        P[i,debut:fin] = b 

    return P 

In [ ]:
def reduction_markovitz(sigma,mu,alpha,K,bit=1,penaly = 0,compact= False):
    """
    From the covariance matrix and the excpected return vector returns the J and H 
    of the Ising mapping 

    ARGS: 
    sigma: covariance matrix 
    mu: excpected return vector
    alpha : aversion constant 
    K : The budget 
    penalty: If we add the constraint in the energy 
    Output: 
    J : 
    H: 
    """
    Bit_max = int(np.log2(K))
    bit = Bit_max
    N = len(mu) # Number of assets considered 
    U = np.ones(bit*N) 
    P = P_aug(N,bit)
    sigma_augmented = P.T @ sigma @ P
    mu_augmented = P.T @ mu 
    I_augmented = P.T @ np.ones(N)

    J = - (alpha/2)* sigma_augmented
    H = (alpha/2) * sigma_augmented @ U - mu_augmented - penaly*I_augmented
    np.fill_diagonal(J,0)

    if not compact: 
        return  J,H
    else: 
        # In here we will follow the methods used in paper 2023
        J_compact = np.zeros((len(J)+1,len(J)+1))
        J_compact[:len(J),:len(J)] = J
        J_compact[:len(J),len(J)] = H 
        J_compact[len(J),:len(J)] = H
        H_compact = np.zeros(len(H)+1)
        # H_compact[:len(H)] = H

        return J_compact,H_compact

In [ ]:
def pumpuing_rate(self, t):
        # return t/100
        if t<lag:
            return 0
        else:
            return 1 * p_last * (t-lag)/(num_iterations-lag)

In [ ]:
def reverse(solution,N,K): 
    solution += 1
    solution /=2
    bit = int(np.log2(K))
    P = P_aug(N,bit)
    return P@solution

In [ ]:
def penalized_trajectory(J,H,s):
    Penality=c/2 P.T @ P
    J=J+Penality
    H=H-2*Penality @ s
    return J,H

In [ ]:
def resolve_optimal_portfolio_trajectory(sigma_list,mu_list)   
    
    for t in range(len(mu_list)):
        J,H = reduction_markovitz(sigma,mu,alpha = 2,K=10,bit=3,penaly= 1,compact=False)
        if t>0:
            J,H  = penalized_trajectory (J,H,states[-1])
        eigs = np.linalg.eigvals(J)
        ksi = 1/(eigs.max())
        p_first = 1-ksi*np.real(eigs.max())
        p_last = 1-ksi*np.real(eigs.min())
        num_iterations = 1000
        lag = 0
        manager = simulation_manager.SimulationManager(step_size=0.5, num_iterations=num_iterations, num_simulations=2, J=J, H=H, pumping_rate=pumpuing_rate, stopping_criterion=0.1, save_states_history=True, save_energies_history=True, n_threads=2, savetofile=False)
        new_states, _, _, _, _, _, _, _, _, _, _ = manager.run_simulation()
        states.append(new_states)
    return states
